# <font color='orange'> Import library </font>

In [ ]:
!pip install -q transformers
#!pip install -q datasets
#!pip install -q albumentations
!pip install -q evaluate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import torch
from torch import nn
import copy
import evaluate

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# <font color='orange'> Read data </font>

In [ ]:
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class ImageSegmentationDataset(Dataset):
  """Image segmentation dataset."""

  def __init__(self, dataset, data_path, transform):
    """
    Args:
        dataset
    """
    self.dataset = dataset
    self.data_path = data_path
    self.transform = transform

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    img_path = os.path.join(self.data_path, self.dataset.iloc[idx, 0])
    label_path = os.path.join(self.data_path, self.dataset.iloc[idx, 1])
    image_label = self.dataset.iloc[idx, 2]

    original_image = Image.open(img_path).convert('RGB')
    original_segmentation_map = Image.open(label_path)

    original_image = np.array(original_image)
    original_segmentation_map = np.array(original_segmentation_map.convert('1'))*1

    transformed = self.transform(image=original_image, mask=original_segmentation_map)
    image, segmentation_map = transformed['image'], transformed['mask']

    # convert to C, H, W
    image = image.transpose(2, 0, 1)

    return image, segmentation_map, original_image, original_segmentation_map, image_label

In [ ]:
def train_val_split(dataset, validation_split=0.2):
    """make dataloader for pytorch training

    Args:
        dataset ([object]): [the dataset object]
        train_batch_size ([int]): [training batch size]
        validation_split (float, optional): [validation ratio]. Defaults to 0.2.

    Returns:
        [type]: [description]
    """
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    #train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_batch_size, shuffle=True,)
    # create test_loader
    #test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_set, test_set

In [ ]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

train_transform = A.Compose([
    A.RandomCrop(width=224, height=224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),
])

test_transform = A.Compose([
    A.Resize(width=224, height=224),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),

])

In [ ]:
datasets = []

for fold in range(0,5):
    df_train = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/all/train_fold_{fold}.csv')
    df_test = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/all/test_fold_{fold}.csv')
    
    #df_train.label = le.transform(df_train.label)
    #df_test.label = le.transform(df_test.label)

    ## train test split
    train_dataset = ImageSegmentationDataset(df_train, data_path='/kaggle/input/ebhi-seg/', transform=train_transform)
    test_dataset = ImageSegmentationDataset(df_test, data_path='/kaggle/input/ebhi-seg/', transform=test_transform)
    
    datasets.append((train_dataset, test_dataset))

In [ ]:
#id2label = {0:'negative', 1:'positive'}
#labels = [id2label[label] for label in np.unique(segmentation_map)]
#print(labels)

# <font color='orange'> Create PyTorch DataLoaders </font>

In [ ]:
from transformers import Mask2FormerImageProcessor

# Create a preprocessor
preprocessor = Mask2FormerImageProcessor(
    ignore_index=-1, reduce_labels=False, do_resize=False,
    do_rescale=False, do_normalize=False,
    num_classes=2,
)

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    inputs = list(zip(*batch))
    images = inputs[0]
    segmentation_maps = inputs[1]
    # this function pads the inputs to the same size,
    # and creates a pixel mask
    # actually padding isn't required here since we are cropping
    batch = preprocessor(
        images,
        segmentation_maps=segmentation_maps,
        return_tensors="pt",
    )

    batch["original_images"] = inputs[2]
    batch["original_segmentation_maps"] = inputs[3]
    batch['imgage_label'] = inputs[4]
    
    return batch

In [ ]:
dataloaders=[]
for i, (train_set, test_set) in enumerate(datasets):
    train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
    dataloaders.append((train_dataloader, test_dataloader))

# <font color='orange'> Verify data (!!) </font>

Next, it's ALWAYS very important to check whether the data you feed to the model actually makes sense. It's one of the main principles of [this amazing blog post](http://karpathy.github.io/2019/04/25/recipe/), if you wanna debug your neural networks.

Let's check the first batch, and its content.

import torch

train_dataloader = dataloaders[0][1]
batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(type(v))
    if isinstance(v, torch.Tensor):
        print(k,v.shape)
    else:
        if isinstance(v[0], str):
            print(k, ':', v[0])
        else:
            print(k,v[0].shape)

# <font color='orange'>  Define segmentation model </font>

Next, we define the model. We equip the model with pretrained weights from the 🤗 hub. We will replace only the classification head. For that we provide the id2label mapping, and specify to ignore mismatches keys to replace the already fine-tuned classification head.

In [ ]:
from transformers import Mask2FormerForUniversalSegmentation
from torch import Tensor

class PolypSegmentationNetwork(nn.Module):
    def __init__(self):
        super(PolypSegmentationNetwork, self).__init__()
        
        # Tạo instance của Mask2FormerForSegmentation
        self.mask2former = Mask2FormerForUniversalSegmentation.from_pretrained(
            "facebook/mask2former-swin-base-coco-panoptic")
        
    
    def forward(self, pixel_values, class_labels=None, mask_labels=None):
        outputs = self.mask2former(
            pixel_values=pixel_values, 
            class_labels=class_labels, 
            mask_labels=mask_labels
        )
        
        # Tính toán đầu ra của lớp linear cuối cùng
        #logits = self.segmentation_head(outputs.last_hidden_state)
        return outputs

# <font color='orange'> Get encoder </font>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from transformers import Mask2FormerForUniversalSegmentation

seg_model = PolypSegmentationNetwork()
seg_model = seg_model.to(device)

In [ ]:
optimizer = torch.optim.Adam(seg_model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2, factor=0.5)

In [ ]:
from tqdm.auto import tqdm
from torch import nn

def segmentation_encoder(model, dataloader, device):
    #metric = evaluate.load("mean_iou")
    model.eval()
    
    feature_maps = []
    image_labels = []
    
    #extract train feature map
    for idx, batch in enumerate(tqdm(dataloader)):
        pixel_values = batch["pixel_values"]
        img_labels = batch["imgage_label"]
        for label in img_labels:
            image_labels.append(label)

        # Forward pass
        with torch.no_grad():
            outputs = model(pixel_values=pixel_values.to(device))
            fmaps = outputs.encoder_last_hidden_state
            
            avgPool = nn.AvgPool2d(kernel_size=(7,7))
            fmaps = avgPool(fmaps)
            shape = fmaps.shape
            fmaps = fmaps.reshape(shape[0], shape[1])
            
            fmaps = fmaps.cpu().detach().numpy()
            for fmap in fmaps:
                feature_maps.append(fmap)
                
    return feature_maps, image_labels

# <font color='orange'>  Build Classification model </font>

## <font color='orange'>Create dataset and dataloader</font>

In [ ]:
#from PIL import Image
import os
import torch

# define a data class
class FeatureDataset:
    def __init__(self, features, labels):
        """Define the dataset for classification problems

        Args:
            feature_maps ([numpy]): [a float numpy contain 50.176 items]
            img_labels ([numpy]): [a int numpy contain 50.176 encoder images label]
            transform : [augmentation methods and transformation of images]
            training (bool, optional): []. Defaults to True.
        """
        self.features = features
        self.labels = labels

    def __getitem__(self, idx):
        inputs = self.features[idx]
        label = self.labels[idx]
        return inputs, label

    def __len__(self):
        return len(self.features)

In [ ]:
def make_loader(dataset, train_batch_size, validation_split=0.2):
    """make dataloader for pytorch training

    Args:
        dataset ([object]): [the dataset object]
        train_batch_size ([int]): [training batch size]
        validation_split (float, optional): [validation ratio]. Defaults to 0.2.

    Returns:
        [type]: [description]
    """
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=train_batch_size, shuffle=True,
    )
    # create test_loader
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_loader, test_loader

from sklearn.model_selection import KFold

for folder in range(5):
    folds = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, test_idx in kf.split(features):
        train_set = FeatureDataset(features[train_idx], img_labels[train_idx])
        test_set = FeatureDataset(features[test_idx], img_labels[test_idx])
        
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=False)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False)

        folds.append((train_loader, test_loader))

In [ ]:
from sklearn import preprocessing

img_labels = ['adenocarcinoma','high-grade in','low-grade in','normal','polyp','serrated adenoma']
le = preprocessing.LabelEncoder()
le.fit(img_labels)

In [ ]:
cls_dataloaders=[]

for fold in range(0,5):
    #load segmentation weight
    seg_model.load_state_dict(torch.load(f"/kaggle/input/seg-model-weight/seg_model/seg_model_fold{fold}.pt"))
    seg_model = seg_model.to(device)
    
    #get segmentation encoder
    train_dataloader, test_dataloader = dataloaders[fold]
    train_features, train_labels = segmentation_encoder(seg_model, train_dataloader, device)
    test_features, test_labels = segmentation_encoder(seg_model, test_dataloader, device)
    train_labels, test_labels = le.transform(train_labels), le.transform(test_labels)
    
    #save encoder
    np.savez(f"train_features_{fold}.npz", features=train_features, img_labels=train_labels)
    np.savez(f"test_features_{fold}.npz", features=test_features, img_labels=test_labels)
           
    #create dataset
    train_set = FeatureDataset(train_features, train_labels)
    test_set = FeatureDataset(test_features, test_labels)
    #create dataloader    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False)
    
    cls_dataloaders.append((train_loader, test_loader))

## <font color='orange'>Metrics and defining the model</font>

In [ ]:
from sklearn import metrics as skmetrics

class Metrics:
    def __init__(self, metric_names):
        self.metric_names = metric_names
        # initialize a metric dictionary
        self.metric_dict = {metric_name: [0] for metric_name in self.metric_names}

    def step(self, labels, preds):
        for metric in self.metric_names:
            # get the metric function
            do_metric = getattr(
                skmetrics, metric, "The metric {} is not implemented".format(metric)
            )
            # check if metric require average method, if yes set to 'micro' or 'macro' or 'None'
            try:
                self.metric_dict[metric].append(
                    do_metric(labels, preds, average="macro")
                )
            except:
                self.metric_dict[metric].append(do_metric(labels, preds))

    def epoch(self):
        # calculate metrics for an entire epoch
        avg = [sum(metric) / (len(metric) - 1) for metric in self.metric_dict.values()]
        metric_as_dict = dict(zip(self.metric_names, avg))
        return metric_as_dict

    def last_step_metrics(self):
        # return metrics of last steps
        values = [self.metric_dict[metric][-1] for metric in self.metric_names]
        metric_as_dict = dict(zip(self.metric_names, values))
        return metric_as_dict

In [ ]:
train_metrics = Metrics(["accuracy_score", "f1_score"])
val_metrics = Metrics(["accuracy_score", "f1_score"])

In [ ]:
#https://www.kaggle.com/code/darthmanav/multilayer-perceptron-fine-tuning-pca-t-sne-mnist
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
                
        self.input_fc = nn.Linear(input_dim, 250)
        self.hidden_fc = nn.Linear(250, 100)
        self.output_fc = nn.Linear(100, output_dim)
        
    def forward(self, x):
        #x = [batch size, height, width]
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        
        #x = [batch size, height * width]
        h_1 = F.relu(self.input_fc(x))
        
        #h_1 = [batch size, 250]
        h_2 = F.relu(self.hidden_fc(h_1))

        #h_2 = [batch size, 100]
        y_pred = self.output_fc(h_2)
        #y_pred = [batch size, output dim]
        
        return y_pred

## <font color='orange'> Training the Model functions </font>

In [ ]:
def train_one_fold_classifier(
    fold,
    model,
    train_loader,
    device,
    optimizer,
    criterion,
    n_epochs = 100
):
    # to track the training loss as the model trains
    train_losses = []
    execute_times = []

    # initialize tracker for minimum validation loss
    train_loss_min = np.Inf # set initial "min" to infinity

    for epoch in range(n_epochs):
        # monitor training loss
        train_loss = 0.0
        start_time = time.time()

        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        for data, target in train_loader:
            data = data.to(device)
            target = target.to(device)
            #print('target shape: ', target.shape)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #print('outputs shape: ', output.shape)
            # calculate the loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()

            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item()*data.size(0)

        train_loss = train_loss/len(train_loader.dataset)
        train_losses.append(train_loss)
        
        #print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss,))

        # save model if validation loss has decreased
        if train_loss <= train_loss_min:
            print('Epoch: {} \tTraining loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(epoch+1, train_loss_min, train_loss))
            torch.save(model.state_dict(), f'clf_model_fold{fold}.pt')
            train_loss_min = train_loss
        
        execute_time = time.time()-start_time
        execute_times.append(execute_time)
        df_cls_model_train = pd.DataFrame(data={
            "time": execute_times
        })
        df_cls_model_train.to_csv(f"train_cls_model_time_fold{fold}.csv", index=True)
    return train_losses

In [ ]:
def predict(model, dataloader, device):
    predicted_values = []
    target_values=[]
    model.eval()
    with torch.no_grad():
        for data, target in dataloader:
            data = data.to(device)
            target = target.to(device)
            #print(target)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1) 
            predicted_values.extend(predicted.cpu().detach().numpy())
            target_values.extend(target.tolist())
    return predicted_values, target_values

In [ ]:
#def load_feature(fold=0):
#    #load train feature
#    aa = np.load(f'/kaggle/working/train_features_{fold}.npz')
#    train_features, train_labels = aa["features"], aa["img_labels"]
#    train_features = train_features.astype("float32")
#    train_labels = le.transform(train_labels)
    
#    #load test feature
#    aa = np.load(f'/kaggle/working/test_features_{fold}.npz')
#    test_features, test_labels = aa["features"], aa["img_labels"]
#    test_features = test_features.astype("float32")
#    test_labels = le.transform(test_labels)
    
#    return (train_features, train_labels), (test_features, test_labels)

In [ ]:
def train_one_fold_pipeline(fold, test_cls_model, dataloaders, device, n_epochs=100):
    train_loader, test_loader = dataloaders[fold]
    #train model
    train_losses = train_one_fold_classifier(fold, test_cls_model, train_loader, device, 
                               cls_optimizer, cls_criterion, n_epochs=n_epochs)
    #predict test data
    test_cls_model.load_state_dict(torch.load(f'/kaggle/working/clf_model_fold{fold}.pt'))
    predicted_values, target_values = predict(test_cls_model, test_loader, device)
    
    return train_losses, predicted_values, target_values

# <font color='orange'>  Inference Fold 0 </font>

In [ ]:
#create new classification model
cls_model = MLP(input_dim=1024, output_dim=6)
cls_model = cls_model.to(device)

# define loss function
cls_criterion = nn.CrossEntropyLoss()
cls_optimizer = torch.optim.Adam(cls_model.parameters(), lr=1e-4)
##optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train_losses, predicted_values, target_values = train_one_fold_pipeline(0, cls_model, cls_dataloaders, device) 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses)
plt.title('Loss Graph')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
report = classification_report(target_values, predicted_values, digits=6,target_names=le.classes_)
print(report)

In [ ]:
cm = confusion_matrix(target_values, predicted_values)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()

# <font color='orange'>  Inference Fold 1 </font>

In [ ]:
#create new classification model
cls_model = MLP(input_dim=1024, output_dim=6)
cls_model = cls_model.to(device)

# define loss function
cls_criterion = nn.CrossEntropyLoss()
cls_optimizer = torch.optim.Adam(cls_model.parameters(), lr=1e-4)
##optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train_losses, predicted_values, target_values = train_one_fold_pipeline(1, cls_model, cls_dataloaders, device) 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses)
plt.title('Loss Graph')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
report = classification_report(target_values, predicted_values, digits=6,target_names=le.classes_)
print(report)

In [ ]:
cm = confusion_matrix(target_values, predicted_values)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()

# <font color='orange'>  Inference Fold 2 </font>

In [ ]:
#create new classification model
cls_model = MLP(input_dim=1024, output_dim=6)
cls_model = cls_model.to(device)

# define loss function
cls_criterion = nn.CrossEntropyLoss()
cls_optimizer = torch.optim.Adam(cls_model.parameters(), lr=1e-4)
##optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train_losses, predicted_values, target_values = train_one_fold_pipeline(2, cls_model, cls_dataloaders, device) 

In [ ]:


##load best model
#test_seg_model = copy.deepcopy(seg_model)
#test_seg_model.load_state_dict(torch.load("/kaggle/input/seg-model-weight/seg_model_fold2.pt"))
#test_seg_model = test_seg_model.to(device)

In [ ]:
#train_losses, predicted_values, target_values = train_one_fold_pipeline(
#    2, test_seg_model, dataloaders, device)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses)
plt.title('Loss Graph')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
report = classification_report(target_values, predicted_values, digits=6,target_names=le.classes_)
print(report)

In [ ]:
cm = confusion_matrix(target_values, predicted_values)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()

# <font color='orange'>  Inference Fold 3 </font>

In [ ]:
#create new classification model
cls_model = MLP(input_dim=1024, output_dim=6)
cls_model = cls_model.to(device)

# define loss function
cls_criterion = nn.CrossEntropyLoss()
cls_optimizer = torch.optim.Adam(cls_model.parameters(), lr=1e-4)
##optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train_losses, predicted_values, target_values = train_one_fold_pipeline(3, cls_model, cls_dataloaders, device) 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses)
plt.title('Loss Graph')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
report = classification_report(target_values, predicted_values, digits=6,target_names=le.classes_)
print(report)

In [ ]:
cm = confusion_matrix(target_values, predicted_values)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()

# <font color='orange'>  Inference Fold 4 </font>

In [ ]:
#create new classification model
cls_model = MLP(input_dim=1024, output_dim=6)
cls_model = cls_model.to(device)

# define loss function
cls_criterion = nn.CrossEntropyLoss()
cls_optimizer = torch.optim.Adam(cls_model.parameters(), lr=1e-4)
##optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train_losses, predicted_values, target_values = train_one_fold_pipeline(4, cls_model, cls_dataloaders, device) 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses)
plt.title('Loss Graph')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
report = classification_report(target_values, predicted_values, digits=6,target_names=le.classes_)
print(report)

In [ ]:
cm = confusion_matrix(target_values, predicted_values)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()